# Lab 1 Exercises for COMP 6321 (Machine Learning)
# Somaye Ghahari
# 40106359

This is a Jupyter Notebook.

When you opened it, the Jupyter server started a new Python interpreter to run the code you type in the cells.

* **To edit a code cell**, click inside its text box. A green border should appear.
* **To run the selected cell**, hit Ctrl+Enter.

Try typing `3+4` into the cell below and run it with Ctrl+Enter. You should see `7` appear in the output.

In [32]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import ast
%matplotlib inline
sns.set_style('whitegrid')
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100

In [33]:
df_credit = pd.read_csv("archive/credits.csv", dtype='unicode')
columns = ['cast', 'crew']

for column in columns:
    # use ast because json data has single quotes in the csv, which is invalid for a json object; it should be " normally
    df_credit[column] = df_credit[column].apply(lambda x: np.nan if pd.isnull(x) else ast.literal_eval(x))


In [34]:
df_credit.head(10)

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': 'Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862
5,"[{'cast_id': 25, 'character': 'Lt. Vincent Han...","[{'credit_id': '52fe4292c3a36847f802916d', 'de...",949
6,"[{'cast_id': 1, 'character': 'Linus Larrabee',...","[{'credit_id': '52fe44959251416c75039da9', 'de...",11860
7,"[{'cast_id': 2, 'character': 'Tom Sawyer', 'cr...","[{'credit_id': '52fe46bdc3a36847f810f797', 'de...",45325
8,"[{'cast_id': 1, 'character': 'Darren Francis T...","[{'credit_id': '52fe44dbc3a36847f80ae0f1', 'de...",9091
9,"[{'cast_id': 1, 'character': 'James Bond', 'cr...","[{'credit_id': '52fe426ec3a36847f801e14b', 'de...",710


In [35]:
credits_flattened = pd.DataFrame(None,None,columns=['movie_id',
                        'actor_1_name','actor_2_name','actor_3_name',\
                        'director_gender','director_name','producer_gender','producer_name','casting_gender',\
                                'casting_name'])

for i,row in df_credit.iterrows():
    # dummy row
    newrow = {'movie_id':np.nan,\
              'actor_1_name':np.nan,'actor_2_name':np.nan,'actor_3_name':np.nan,\
            'director_gender':np.nan,'director_name':np.nan,'producer_gender':np.nan,\
            'producer_name':np.nan,'casting_gender':np.nan,'casting_name':np.nan}
    
    # fill movie id
    newrow['movie_id'] = int(row['id'])
    
    # fill cast
    count=1
    for item in row['cast']:
        if count==4:
            break
        if 'gender' in item and 'name' in item:
            newrow['actor_'+str(count)+'_name'] = item['name']
            count += 1

    # fill crew
    director=0
    producer=0
    casting=0
    for item in row['crew']:
        if director and producer and casting:
            break
        if 'job' in item and item['job'] in ['Director','Producer','Casting'] and 'gender' in item and 'name' in item:
            if item['job'] == 'Director' and not director:
                newrow['director_gender'] = item['gender']
                newrow['director_name'] = item['name']
                director =  1
            elif item['job'] == 'Producer' and not producer:
                newrow['producer_gender'] = item['gender']
                newrow['producer_name'] = item['name']
                producer =  1
            elif item['job'] == 'Casting' and not casting:
                newrow['casting_gender'] = item['gender']
                newrow['casting_name'] = item['name']
                casting =  1

    credits_flattened = credits_flattened.append(newrow,ignore_index=True)

In [36]:
credits_flattened.head(10)

,movie_id,actor_1_name,actor_2_name,actor_3_name,director_gender,director_name,producer_gender,producer_name,casting_gender,casting_name
0,862,Tom Hanks,Tim Allen,Don Rickles,2,John Lasseter,1,Bonnie Arnold,NaN,NaN
1,8844,Robin Williams,Jonathan Hyde,Kirsten Dunst,2,Joe Johnston,2,Scott Kroopf,0,Nancy Foy
2,15602,Walter Matthau,Jack Lemmon,Ann-Margret,2,Howard Deutch,NaN,NaN,NaN,NaN
3,31357,Whitney Houston,Angela Bassett,Loretta Devine,2,Forest Whitaker,0,Ronald Bass,NaN,NaN
4,11862,Steve Martin,Diane Keaton,Martin Short,2,Charles Shyer,1,Nancy Meyers,NaN,NaN
5,949,Al Pacino,Robert De Niro,Val Kilmer,2,Michael Mann,2,Art Linson,1,Bonnie Timmermann
6,11860,Harrison Ford,Julia Ormond,Greg Kinnear,2,Sydney Pollack,2,Sydney Pollack,2,David Rubin
7,45325,Jonathan Taylor Thomas,Brad Renfro,Rachael Leigh Cook,2,Peter Hewitt,NaN,NaN,NaN,NaN
8,9091,Jean-Claude Van Damme,Powers Boothe,Dorian Harewood,2,Peter Hyams,2,Moshe Diamant,NaN,NaN
9,710,Pierce Brosnan,Sean Bean,Izabella Scorupco,2,Martin Campbell,1,Barbara Broccoli,1,Debbie McWilliams


In [4]:
def create_comparison_database(name, value, x, no_films):
    
    comparison_df = df.groupby(name, as_index=False)
    
    if x == 'mean':
        comparison_df = comparison_df.mean()
    elif x == 'median':
        comparison_df = comparison_df.median()
    elif x == 'sum':
        comparison_df = comparison_df.sum() 
    
    # Create database with either name of directors or actors, the value being compared i.e. 'gross',
    # and number of films they're listed with. Then sort by value being compared.
    name_count_key = df[name].value_counts().to_dict()
    comparison_df['films'] = comparison_df[name].map(name_count_key)
    comparison_df.sort_values(value, ascending=False, inplace=True)
    comparison_df[name] = comparison_df[name].map(str) + " (" + comparison_df['films'].astype(str) + ")"
   
    # create a Series with the name as the index so it can be plotted to a subgrid
    comp_series = comparison_df[comparison_df['films'] >= no_films][[name, value]][10::-1].set_index(name).ix[:,0]
    
    return comp_series

In [27]:
df_meta = pd.read_csv("archive/movies_metadata.csv")




/Users/somayeghahari/opt/miniconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [28]:
df_meta.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0


In [15]:
create_comparison_database('director_name','gross','sum', 0).plot(kind='barh', color='#006600')

KeyError: 'director_name'